In [1]:
%cd ..

import numpy as np
import pandas as pd
import rx
from rx import operators
from rx import scheduler
from rx import subject

import logging
import datetime

from configs.config import GlobalConfig
GlobalConfig.initialize_global_configuration("configs/demo.json")

from utils.logging import configure_logger_to_output
logging_output = configure_logger_to_output(level=logging.INFO)

C:\Users\wuyua\PycharmProjects\RxExperiments


# Description


This file contains the demonstration to use the notebook as the experiment terminal. It contains
- Image acquisition
- Temperature measurement
- FBRM reading
- RAMAN reading
- FP50 reading
- FP50 control
- Event logging widget

## Shared initialization

In [2]:
from utils.mqtt_wrapper import MQTTClientWrapper
from source.harvesters_source import HarvestersSource
from sink.save_image_sink import SaveImageSink
from sink.save_data_sink import SaveDataSink
from sink.visualization_sink import JupyterImageSink, PlotlyVisualizationSink
from datamodel.image import Image
from operators import data_framer, lttb
from plotly import graph_objects as go
from controls.camera_controller import CameraControl, CameraControlCommand
from controls.fp50 import FP50Command, FP50Control
from controls.modbus_pump import ModbusPumpControl, ModbusPumpControlCommand
from source.ds18_source import DS18Source
from source.fbrm_source import FBRMSource
from source.raman_source import RamanSource
import ipywidgets as widgets


In [3]:
# MQTT connection
client = MQTTClientWrapper("experiment_demo")
client.connect("workstation.local")
client.loop_start()

### Image analysis

In [4]:
# Image acquisition
image_acquisiton_enabled = False

# auto adjust will kick in if the average brightness is out of brightness_target +- deviation
brightness_target = 220
deviation = 10

image_source = HarvestersSource().pipe(operators.filter(lambda x: image_acquisiton_enabled), operators.share())
image_save_sink = SaveImageSink()
camera_control = CameraControl()
current_exposure = 2

# save image logic
image_save_subscription = image_source.subscribe(image_save_sink)

# camera controller logic
image_mean_brightness_source = image_source.pipe(
    operators.map(lambda x: x.image.mean())
)

def filter_brightness(x):
    hit_rate = np.bitwise_or(x > brightness_target + deviation, x < brightness_target - deviation).mean()
    mean_brightness = x.mean()
    return hit_rate, mean_brightness

def prepare_camera_control_command(x):
    current_exposure = prepare_camera_control_command.current_exposure 
    current_exposure = current_exposure * brightness_target / x[1]
    prepare_camera_control_command.current_exposure = current_exposure
    logging.info(f"exposure auto control: brightness={x[1]}; exposure={current_exposure}")
    return CameraControlCommand().set_exposure(current_exposure)
prepare_camera_control_command.current_exposure = current_exposure # inject static variable

camera_auto_control_subscription = image_mean_brightness_source.pipe(
    operators.buffer_with_count(10), # buffer images
    operators.map(lambda x: np.asarray(x)), # convert into array.
    operators.map(filter_brightness),
    operators.filter(lambda x: x[0] >= 0.8), # only adjust if at least 4 out of 5 are not ok
    operators.map(prepare_camera_control_command)
).subscribe(camera_control)

# enable fan and trigger at the beginning
camera_control.on_command(CameraControlCommand().set_exposure(current_exposure).set_power(True).set_trigger(True))

# dataframed source
image_brightness_branch = image_mean_brightness_source.pipe(
    operators.sample(2),
    operators.map(lambda x: pd.DataFrame(data=[{"value": x}], index=[datetime.datetime.now()]))
)

# save file logic
mean_brightness_save_sink = SaveDataSink("image_brightness", "value")
mean_brightness_save_subscription = image_brightness_branch.subscribe(mean_brightness_save_sink)

In [5]:
# Front end panels
# camera visuallization logic
camera_image_sink = JupyterImageSink(name="Camera")
camera_visuallization_subscription = image_source.pipe(operators.sample(1)).subscribe(camera_image_sink)

# camera brightness visuallization logic
mean_brightness_sink = PlotlyVisualizationSink(name="Mean brightness")

mean_brightness_visuallization_subscription = image_brightness_branch.pipe(
    data_framer.data_framer(),
    operators.map(lambda x: [go.Scatter(x=x.index, y=x.value)])
).subscribe(mean_brightness_sink)

### Pump control

In [6]:
# back end
pump_control = ModbusPumpControl()
pump_control_dataframe = pump_control.pipe(
    operators.map(
        lambda x: pd.DataFrame(
            index=[datetime.datetime.now()], 
            data=[{"slurry": x.slurry_pump, "clear": x.clear_pump}]
        )
    )
)

pump_control_save = SaveDataSink("pump", "data")
pump_control_save_subscription = pump_control_dataframe.subscribe(pump_control_save)

## Water bath and temperature control

In [7]:
# Backend configuration

temperature_source = DS18Source(client).pipe(operators.share())
fp50_control = FP50Control(client)

fp50_dataframe_source = rx.combine_latest(fp50_control.pipe(operators.share()), temperature_source).pipe(
    operators.sample(1), # sample time
    operators.map(lambda x: pd.DataFrame(index=[datetime.datetime.now()], data=[{ 
        "power": x[0].power, 
        "setpoint": x[0].setpoint,
        "internal": x[0].temperature,
        "temperature": x[1],
    }])),
)

# save file logic
fp50_save_sink = SaveDataSink("fp50", "data")
fp50_save_subscription = fp50_dataframe_source.subscribe(fp50_save_sink)

In [8]:
# front end panels
fp50_visualization_sink = PlotlyVisualizationSink(name="FP50")

fp50_visuallization_subscription = fp50_dataframe_source.pipe(
    operators.sample(5),
    data_framer.data_framer(),
    operators.map(lambda x: [
        go.Scatter(x=x.index, y=x[col], yaxis=f"y{2 if col == 'power' else 1}", name=col)
    for col in x.columns])
).subscribe(fp50_visualization_sink)
fp50_visualization_sink.figure.update_layout(
    yaxis={"title": "$Temperature (^\circ C)$", "side": "left", "range": [10, 70]},
    yaxis2 = {"title": "Power", "side": "right", "overlaying": "y", "range": [-100, 100]},
    xaxis={"title": "Time"}
)
;

''

### FBRM

In [9]:
# backend configuration
fbrm_source = FBRMSource(client, "fbrm").pipe(operators.share())

fbrm_dataframe_source = fbrm_source.pipe(
    operators.map(
        lambda x: pd.DataFrame(
            index=[datetime.datetime.now()], 
            data=[dict(zip(x["sizes"], x["counts"]))],
        )
    )
)

# save file logic
fbrm_save_sink = SaveDataSink("fbrm", "data")
fbrm_save_subscription = fbrm_dataframe_source.subscribe(fbrm_save_sink)

In [10]:
# front end configuration
fbrm_visuallization = PlotlyVisualizationSink(name="FBRM")
fbrm_visuallization_subscription = fbrm_source.pipe(
    operators.map(lambda x: [go.Scatter(x=x["sizes"], y=x["counts"], )])
).subscribe(fbrm_visuallization)

fbrm_visuallization.figure.update_layout(xaxis={"type":"log", "title":"$Size (\mu m)$"}, yaxis={"title": "Count"})

fbrm_count_visuallization = PlotlyVisualizationSink(name="FBRM statistics")
fbrm_count_visuallization_subscription = fbrm_source.pipe(
    operators.map(lambda x: pd.DataFrame(index=[datetime.datetime.now()], data=[{
        "total": np.sum(x["counts"])
    }])),
    data_framer.data_framer(),
    operators.map(lambda x: [go.Scatter(x=x.index, y=x.total, )])
).subscribe(fbrm_count_visuallization)
;

''

# Raman

In [11]:
# backend configuration
raman_source = RamanSource(client, "raman").pipe(operators.share())

raman_dataframe_source = raman_source.pipe(
    operators.map(
        lambda x: pd.DataFrame(
            index=[datetime.datetime.now()], 
            data=[dict(zip(x["wave_number"], x["count"]))],
        )
    )
)

# save file logic
raman_save_sink = SaveDataSink("raman", "data")
raman_save_subscription = raman_dataframe_source.subscribe(raman_save_sink)

In [12]:
# front end configuration
raman_visuallization = PlotlyVisualizationSink(name="Raman")
raman_visuallization_subscription = raman_source.pipe(
    operators.map(lambda x: [go.Scatter(x=x["wave_number"], y=x["count"], )])
).subscribe(raman_visuallization)

raman_visuallization.figure.update_layout(xaxis={"autorange": "reversed", "title":"$Wave number (cm^{-1})$"}, yaxis={"title": "Count"})

;

''

## Organize widgets

In [13]:
from ipywidgets import Layout, Box
# logging panel
logging_output.layout = Layout(border="solid", width="100%", height="200px", overflow="scroll")

# image panel
image_panel_layout = Layout(display="flex", flex_flow="row", border="solid", width="100%")
mean_brightness_figure = Box(children=[mean_brightness_sink.figure], layout=Layout(flex="1 1 0%", width="auto"))
camera_image_sink.figure.layout = Layout(flex="1 1 0%", width="65%")
image_panel = Box(children=[mean_brightness_sink.figure, camera_image_sink.figure], layout=image_panel_layout)


# temperature panel
temperature_panel = Box(children=[fp50_visualization_sink.figure], layout=
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)

# FBRM panel
fbrm_panel = Box(children=[fbrm_visuallization.figure, fbrm_count_visuallization.figure], layout= 
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)

# Raman Panel
raman_panel = Box(children=[raman_visuallization.figure], layout=
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)


# control panel
event_logger_text = widgets.Text(
    description='Event annotation:',
    layout=Layout(width="100%"),
)
event_logger_text.on_submit(lambda x: logging.getLogger("annotation").info(x.value))


control_panel = Box(children=[event_logger_text], layout=Layout(border="solid", width="100%"))

# wrap up
panel = widgets.VBox([logging_output, image_panel, temperature_panel, fbrm_panel, raman_panel, control_panel])

display(panel)

## Stage 1: start up
- RPM 200
- heat up to 50 $^\circ C$
- hold until FBRM reading is zero

In [14]:
logging.getLogger("stage_reporter").info("stage 1 starts.")

threshold = 30

fp50_control.on_command(FP50Command(60))
stage1_stop = subject.Subject()


stage1_output = widgets.Output()

@stage1_output.capture()
def check_fbrm_count(x: float):
    if x < threshold:
        print(f"Current FBRM count = {x}. Please initiate next step!")
        
fbrm_source.pipe(
    operators.map(lambda x: np.sum(x["counts"])),
    operators.take_until(stage1_stop),
).subscribe(check_fbrm_count, logging.error)

stage1_output

Output()

Stage 1 clean up code

In [18]:
logging.getLogger("stage_reporter").info("stage 1 ends.")

stage1_stop.on_next(True)
stage1_stop.on_completed()

## Stage 2: Fast cool to solubility
- fp50 set to solubility (40 celsius)
- monitor FBRM reading: do not increase

In [19]:
logging.getLogger("stage_reporter").info("stage 2 starts.")
fp50_control.on_command(FP50Command(40))

stage 2 clean up code

In [20]:
logging.getLogger("stage_reporter").info("stage 2 ends.")

## Stage 3: fast cooling to metastable zone
- cool down to 32 degc (8 degc metastable zone)
Then,
- start pipe heating
- monitor the fbrm reading
- enable pump to 200
Then, 
- start wet milling for 1 minute, check fbrm reading after settle down. 
- if fbrm reading is below 10000, restart milling for another 1 min
- if above 15000, heat up

In [21]:
logging.getLogger("stage_reporter").info("stage 3 starts.")
fp50_control.on_command(FP50Command(32))

When temperature reaches desired T and stable, start piping heating, run the code to start camera, pump on.


In [22]:
logging.getLogger("stage_reporter").info("stage 3 camera start up. Heater on.")
pump_control.on_command(ModbusPumpControlCommand().set_slurry_pump(-200))
image_acquisiton_enabled = True


Start milling for one minute, then stop and let the bubble to settle down (power 22)

In [24]:
logging.getLogger("stage_reporter").info("stage 3 start wet milling")
def timer_triggered(x):
    logging.getLogger("stage_reporter").info("stage 3 stop wet milling")

rx.timer(60.0).subscribe(timer_triggered)


In [25]:
logging.getLogger("stage_reporter").info("stage 4 start cooling")

rx.interval(1.0).pipe(
    operators.map(lambda x: 30 - 0.2*x/60), # 30-20, 0.2 degc/min, 
    operators.take_while(lambda x: x >= 20),
    operators.map(lambda x: FP50Command(x)),
).subscribe(fp50_control)

# Conclusion notebook
Agglomeration a lot! fbrm 3000 not enough start
